## Building an Autonomous Web Agents
Multion web Agent

In [1]:
# Let's start by intializing multionn agent and we load the multion key from the utils.

import os
from multion.client import MultiOn

from utils import (get_multi_on_api_key,
                    visualizeSession,
                    MultiOnDemo,
                    SessionManager,
                    ImageUtils,
                    display_step_header)

In [2]:
multion_api_key = get_multi_on_api_key()
multion = MultiOn(api_key=multion_api_key, base_url=os.getenv("DLAI_MULTION_BASE_URL"))

In [4]:
# Intialize the MuiltOn Client

class MultiOnClient:
    """A simplified client for the MultiOn API"""
    
    def __init__(self, multion):
        """Initialize the MultiOn client with the API key"""
        # Use the actual MultiOn class from the imported module
        self.client = multion
        self.session_id = None
        self.current_url = None
        self.screenshot = None
        
    #Creating a session for the new agent
    def create_session(self, url):
        """Create a new agent session"""
        session = self.client.sessions.create(url=url,
                                              include_screenshot=True)
        self.session_id = session.session_id
        self.current_url = session.url
        self.screenshot = session.screenshot
        return session

    #close the current session
    def close_session(self):
        """Close the current session"""
        if self.session_id:
            self.client.sessions.close(self.session_id)
            self.session_id = None

    def list_sessions(self):
        """List all active sessions"""
        return self.client.sessions.list()

    def close_all_sessions(self):
        """Close all open sessions"""
        sessions = self.list_sessions()
        for session in sessions.session_ids:
            self.client.sessions.close(session)

    #navigate to the url
    def navigate_to_url(self, url):
        """Navigate to a URL in the current session"""
        if not self.session_id:
            return self.create_session(url)

        response = self.client.sessions.step(
            session_id=self.session_id,
            cmd=f"GO TO URL {url}",
            include_screenshot=True,
            mode="standard"
        )

        self.current_url = response.url
        self.screenshot = response.screenshot
        return response
    
    #Execute the task in agent session
    def execute_task(self, task):
        """Execute a task in the current agent session"""
        if not self.session_id:
            raise ValueError("No active session. Call create_session first.")
            
        response = self.client.sessions.step(
            session_id=self.session_id,
            cmd=(
                f"""IMPORTANT: DO NOT ASK THE USER ANY QUESTIONS. 
                All the necessary information is already provided 
                and is on the current Page.\n
                Complete the task to the best of your abilities.\n\n
                Task:\n\n{task}"""
            ),
            include_screenshot=True,
        )
        
        self.current_url = response.url
        self.screenshot = response.screenshot
        
        return response
    
# Here you can give the specific task which you want your agent to follow.

In [5]:
multionClient = MultiOnClient(multion)

#### Example 1:
We ask agent to execute task based on each steps.

In [6]:
instruction = "get list all the social media profile from the github homepage"
url = "https://github.com/MansiMore99"
MAX_STEPS = 10
MAX_IMAGE_WIDTH = 500

In [7]:
session = multionClient.create_session(url)

In [8]:
step = 0
while session.status == "CONTINUE" and step < MAX_STEPS:
    display_step_header(step)
    visualizeSession(session, max_image_width=MAX_IMAGE_WIDTH)
    session = multionClient.execute_task(instruction)
    step += 1

In [9]:
# let's use Visualizesession to get final response from the agent

visualizeSession(session, max_image_width=MAX_IMAGE_WIDTH)

### Example 2: MulitOn Browser UI
let's create multion Broswer UI

In [10]:
# Give specific instructions to the agent which it can follow throught the steps.

sessionManager = SessionManager(url, multionClient)
name = "Mansi More"
blog = "Agentforce: Build Your Own Salesforce AI Agent in Just 15 Minutes!"
action_engine = None
instructions = [f"Open the Github homepage of {name}",
                f"""Find the medium link and from her github profile, Open first article {blog} and summarize it"""
               ]

In [11]:
# let's have Mliton broswer demo

demo = MultiOnDemo(url, 
                   sessionManager, 
                   multionClient, 
                   instructions, 
                   action_engine)

In [ ]:
demo.create_demo()

* Running on local URL:  https://0.0.0.0:7860

Could not create share link. Missing file: /usr/local/lib/python3.11/site-packages/gradio/frpc_linux_amd64_v0.3. 

Please check your internet connection. This can happen if your antivirus software blocks the download of this file. You can install manually by following these steps: 

1. Download this file: https://cdn-media.huggingface.co/frpc-gradio-0.3/frpc_linux_amd64
2. Rename the downloaded file to: frpc_linux_amd64_v0.3
3. Move the file to this location: /usr/local/lib/python3.11/site-packages/gradio


In [ ]:
# You can see, it does not follow a particular structure. 
# Because, it more of a conversational assistant, 
# where it follows and answers as more a conversation.
# But, if we give it an instruction: "Give a structured output like JSON on Markdown"
# it would've followed the results.


### Congratulations!!!!

You just have built, your Autnomous AI Agent on MultiOn browser.